In [ ]:
"""
=============================================================================
             RK4 + PHYSICS-INFORMED NEURAL NETWORK (PINN) SIMULATOR
=============================================================================

[1] USER GUIDE & INSTRUCTIONS
-----------------------------------------------------------------------------
This script performs a two-stage simulation of radioactive dispersion in the ocean:
  Stage 1: PHYSICS ENGINE (RK4). Solves the differential equations for water
           flow using the Runge-Kutta 4 method (high precision).
  Stage 2: AI EMULATOR (Neural Network). Trains a neural network to "learn"
           the physics from Stage 1 and generate a continuous prediction map.

HOW TO OPERATE:
1. Run the script entirely. It is fully automated.
   - It will first compute the physics (Simulation).
   - Then it will train the AI (you will see "Epoch" loss logs).
   - Finally, it will display a Heatmap of the result.

REQUIRED LIBRARIES:
You must have the following Python libraries installed:
   pip install numpy matplotlib torch

Code Running Application:
Google Colab is recommended to run this code. You can download the file and
open it through Google Colab.

LIMITATIONS:
1. 1D APPROXIMATION: This model simulates a 1-dimensional "flow channel"
   extending 10,000 km from the source. It assumes uniform mixing in the
   width/depth directions.
2. SOURCE TERM: The source (Q) is modeled as a constant release for the first
   30 years. Changing 'T_max' requires adjusting the 'source' logic manually.
3. NEURAL NETWORK: The accuracy depends on training. If 'Loss' remains high
   (>0.1), re-run the script to re-initialize the random weights. No guarantee
   that you can receive the exact same result due to Neural Network.

*** Gemini is used to generate the explanation and the guidelines for clarity.
=============================================================================
"""

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# ==========================================
# PART 1: RK4 PHYSICS ENGINE (Water Only)
# ==========================================
# EXPLANATION: This function defines the "Rules of Physics" (Differential Equations).
# It calculates how concentration changes based on:
# 1. Source: New material entering (Pipe discharge).
# 2. Advection: Material moving with the current (u/dx).
# 3. Decay: Material disappearing due to radioactivity.
def ocean_derivatives(t, concentrations, params):
    N, u, dx, decay = params['N'], params['u'], params['dx'], params['decay']
    dCdt = np.zeros(N)

    # Source Term: massive release (Q) for the first 30 years.
    source = params['Q'] / (dx * 1000) if t <= 30 else 0

    # Tank 0 (Boundary Condition - where the pipe is)
    dCdt[0] = source - (u/dx)*concentrations[0] - decay*concentrations[0]

    # Tanks 1 to N-1 (The open ocean)
    C_prev, C_curr = concentrations[:-1], concentrations[1:]
    dCdt[1:] = (u/dx)*C_prev - (u/dx)*C_curr - decay*C_curr

    return dCdt

# EXPLANATION: This function solves the math defined above.
# It uses Runge-Kutta 4 (RK4), a standard method for high-accuracy simulations.
# It steps through time (dt = 0.05 years) to build a history of the ocean.
def generate_rk4_data():
    params = {'N': 60, 'u': 2800.0, 'dx': 170.0, 'decay': 0.056, 'Q': 50000.0}
    dt = 0.05
    T_max = 30.0
    time_vec = np.linspace(0, T_max, int(T_max/dt))

    state = np.zeros(params['N'])
    history = np.zeros((len(time_vec), params['N']))

    for i, t in enumerate(time_vec[:-1]):
        history[i] = state
        # --- RK4 Integration Steps (k1, k2, k3, k4) ---
        k1 = ocean_derivatives(t, state, params)
        k2 = ocean_derivatives(t + 0.5*dt, state + 0.5*dt*k1, params)
        k3 = ocean_derivatives(t + 0.5*dt, state + 0.5*dt*k2, params)
        k4 = ocean_derivatives(t + dt, state + dt*k3, params)
        state = state + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    history[-1] = state
    return time_vec, history, params

print("1. Running RK4 Simulation...")
rk4_time, rk4_data, params = generate_rk4_data()

# EXPLANATION: A helper function to "lookup" the ground truth value
# from the RK4 simulation given a specific distance and time.
def get_real_value(dist_km, t_year):
    max_dist = params['N'] * params['dx']
    if t_year <= 0 or dist_km > max_dist: return 0.0
    t_idx = int(t_year / 0.05)
    if t_idx >= len(rk4_time): t_idx = -1
    tank_idx = int(dist_km / params['dx'])
    if tank_idx >= params['N']: return 0.0
    return rk4_data[t_idx, tank_idx]

# ==========================================
# PART 2: TRAIN NEURAL NETWORK
# ==========================================
print("2. Training Neural Network...")
# EXPLANATION: We generate random points to teach the AI.
# The AI tries to predict the output of 'get_real_value' (the physics).

n_samples = 5000
x_train = np.random.uniform(0, 10000, n_samples)
t_train = np.random.uniform(0.1, 30.0, n_samples)
y_train = np.array([get_real_value(d, t) for d, t in zip(x_train, t_train)])

# EXPLANATION: Normalization.
# Neural Networks learn best when inputs are roughly between -1 and 1.
# We subtract the mean and divide by standard deviation.
x_mean, x_std = x_train.mean(), x_train.std()
t_mean, t_std = t_train.mean(), t_train.std()
y_max = y_train.max()

X_tensor = torch.tensor(np.column_stack(((x_train-x_mean)/x_std, (t_train-t_mean)/t_std)), dtype=torch.float32)
Y_tensor = torch.tensor(y_train/y_max, dtype=torch.float32).view(-1, 1)

# EXPLANATION: The "Brain" of the AI.
# 2 Inputs (Distance, Time) -> Hidden Layers -> 1 Output (Concentration).
# Tanh() is used because physics curves are smooth/wavy.
model = nn.Sequential(
    nn.Linear(2, 64),
    nn.Tanh(),
    nn.Linear(64, 64),
    nn.Tanh(),
    nn.Linear(64, 1),
    nn.ReLU()
)
optimizer = optim.Adam(model.parameters(), lr=0.005)

for i in range(2000):
    optimizer.zero_grad()
    loss = nn.MSELoss()(model(X_tensor), Y_tensor)
    loss.backward()
    optimizer.step()
    if i % 200 == 0: print(f"   Epoch {i}, Loss: {loss.item():.6f}")

# ==========================================
# PART 3: GENERATE & PLOT HEATMAP (Standard Bq/L)
# ==========================================
print("3. Generating Visualization...")

# EXPLANATION: Now we ask the trained AI to predict the concentration
# for every point on a grid (200x200 pixels) to create a smooth map.

dist_vals = np.linspace(0, 10000, 200)
time_vals = np.linspace(0, 30, 200)
D_grid, T_grid = np.meshgrid(dist_vals, time_vals)

d_flat, t_flat = D_grid.flatten(), T_grid.flatten()
inputs = np.column_stack(((d_flat - x_mean)/x_std, (t_flat - t_mean)/t_std))

with torch.no_grad():
    preds = model(torch.tensor(inputs, dtype=torch.float32)).numpy().flatten()

# --- UNIT CONVERSION LOGIC ---
# Standard Peak Assumption: ~120 Bq/L
# No multiplier needed for standard Bq/L
REAL_PEAK_Bq_L = 120.0

# Scale the output
Z_grid = ((preds * y_max) * (REAL_PEAK_Bq_L / y_max)).reshape(D_grid.shape)

plt.figure(figsize=(10, 6))

# Plotting
plt.imshow(Z_grid, aspect='auto', origin='lower', cmap='jet',
           extent=[0, 10000, 0, 30], vmin=0, vmax=np.max(Z_grid))

cb = plt.colorbar()
# *** UPDATED LABEL: Bq/L ***
cb.set_label('Water Concentration (Bq/L)', rotation=270, labelpad=20)

plt.xlabel('Distance (km)')
plt.ylabel('Time (Years)')
plt.title(f'RK4 + AI Dispersion Model')
plt.grid(alpha=0.2)

if np.max(Z_grid) < 1e-5:
    print("Warning: Plot is empty. Neural Network prediction is too low.")
else:
    print(f"Success! Max Concentration: {np.max(Z_grid):.2f} Bq/L")

plt.show()

In [ ]:
"""
=============================================================================
             PART 4: STATISTICAL EVALUATION & VALIDATION
=============================================================================

[1] SCIENTIFIC OBJECTIVE
-----------------------------------------------------------------------------
In scientific computing, we cannot trust a Neural Network blindly. We must
quantify its error compared to the "Ground Truth" (the RK4 Physics Engine).

This module performs two critical tests:
1. PARITY PLOT (Linear Regression Analysis):
   - Checks if AI predictions match the Physics Truth 1:1.
   - Ideal Result: All blue dots fall exactly on the red dashed line.
   - Metric: R-Squared (R²) score. 1.0 = Perfect Physics Match.

2. ERROR HISTOGRAM (Residual Analysis):
   - Checks the distribution of errors.
   - Ideal Result: A "Bell Curve" centered tightly at 0.
   - Metric: Mean Squared Error (MSE). Lower is better.

[2] PREREQUISITES
-----------------------------------------------------------------------------
- The variable 'model' must be a trained PyTorch neural network. You must run
the first part of the script to train the AI and then initiate this program. If
not, the code will not run because it is based on the previous result. If run
time lost happened, you must rerun the first part and then initiate this part.
- The function 'get_real_value(d, t)' must be active.
- Normalization stats (x_mean, x_std, etc.) must exist from the training step.

Code Running Application:
Google Colab is recommended to run this code. You can download the file and
open it through Google Colab.

*** Gemini is used to generate the explanation and the guidelines for clarity.
=============================================================================
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import torch

# ==========================================
# 4. MODEL EVALUATION (Accuracy Analysis)
# ==========================================
print("Generating Accuracy Report...")

# A. Generate Test Data (Unseen Points)
# EXPLANATION: We generate NEW random points that the AI has never seen before.
# This ensures we are testing "Generalization" (learning the rule),
# not "Memorization" (cheating).
n_test = 2000
x_test = np.random.uniform(0, 10000, n_test)
t_test = np.random.uniform(0.1, 30, n_test)

# B. Get Truth (Physics) vs Prediction (AI)
# EXPLANATION:
# 1. We ask the Physics Engine (RK4) for the "Real Answer" (y_true).
# 2. We ask the AI Model for its "Guess" (preds_norm).
#    * Crucial: We must normalize the inputs before feeding them to the AI,
#      because the AI was trained on normalized data (0-1 range).
y_true_test = np.array([get_real_value(d, t) for d, t in zip(x_test, t_test)])
inputs_test = np.column_stack(((x_test-x_mean)/x_std, (t_test-t_mean)/t_std))

with torch.no_grad():
    preds_norm = model(torch.tensor(inputs_test, dtype=torch.float32)).numpy().flatten()

y_pred_test = preds_norm * y_max # Scale back to real units (Bq/L)

# C. Calculate Statistics
# EXPLANATION:
# - R2 Score: 1.0 means the AI captures 100% of the physics behavior.
# - MSE: The average squared difference between Truth and Prediction.
# - Residuals: The raw error for each specific point (Prediction - Truth).
r2 = r2_score(y_true_test, y_pred_test)
mse = mean_squared_error(y_true_test, y_pred_test)
residuals = y_pred_test - y_true_test

# D. Plot Results
# EXPLANATION: Creates a professional two-panel scientific figure.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Parity Plot
# Shows correlation. If the model is perfect, y = x (Red Line).
ax1.scatter(y_true_test, y_pred_test, alpha=0.5, s=10, c='blue', label='Test Data')
max_val = max(y_true_test.max(), y_pred_test.max())
ax1.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Fit')
ax1.set_title(f'Model Accuracy (Parity Plot)\n$R^2$ Score = {r2:.4f}')
ax1.set_xlabel('Physics Ground Truth (Normalized)')
ax1.set_ylabel('Neural Network Prediction (Normalized)')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Error Histogram
# Shows bias. We want the green bars to be tall and centered at 0 (Red Line).
# If the bars are shifted left/right, the AI has a "systematic bias".
ax2.hist(residuals, bins=50, color='green', alpha=0.7, edgecolor='black')
ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
ax2.set_title(f'Error Distribution\nMSE = {mse:.4e}')
ax2.set_xlabel('Prediction Error')
ax2.set_ylabel('Frequency')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()